In [6]:
import pandas as pd
import numpy as np

df_to_view = pd.read_csv("State_Actual.csv")
df_to_view.head()

,Unnamed: 0,State,id,Year,State_Minimum_Wage,Federal_Minimum_Wage,CPI_Average
0,0,Alabama,1,1968,0.00000,1.15,34.8
1,1,Alaska,2,1968,2.10000,1.15,34.8
2,2,Arizona,4,1968,0.46800,1.15,34.8
3,3,Arkansas,5,1968,0.15625,1.15,34.8
4,4,California,6,1968,1.65000,1.15,34.8


In [8]:
import altair as alt
import pandas as pd
from vega_datasets import data
from IPython.display import display, HTML

display(HTML("""
<style>
form.vega-bindings {
    position: absolute;
    left: 50px;
    top: 20px;
}
</style>
"""))

states = alt.topo_feature(data.us_10m.url, 'states')
full_df =  df_to_view
fontsize = 20
year_slider = alt.binding_range(min=1968, max=2022, step=1)
slider_selection = alt.selection_single(bind=year_slider, fields=['Year'], name='Select', init={'Year': 2022})
state_selection = alt.selection_multi(fields=['State'], init=[{'State': 'Alabama'}])

 
choropleth = (alt.Chart(full_df, title="Actual Dollars Minimum Wage").mark_geoshape(
    #   state borders stroke and strokeWidth
    stroke='black',
    strokeWidth=1
    ).encode(
    #    more color schemes can be found at https://vega.github.io/vega/docs/schemes/
    color=alt.Color('State_Minimum_Wage:Q', scale=alt.Scale(scheme='bluepurple'), sort='descending'),
    tooltip=[alt.Tooltip('State_Minimum_Wage',format='$'), alt.Tooltip('Federal_Minimum_Wage',format='$')]
    ).add_selection(
        slider_selection,
        state_selection
    ).transform_filter(
        slider_selection
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(states, key='id', fields=['type', 'properties', 'geometry'])
).project('albersUsa'
).properties(
        width=1100,
        height=600
    )
)

state_line = (alt.Chart(full_df).mark_line(color='purple', strokeWidth=3).encode(
    x= 'Year:O',
    y = alt.Y('State_Minimum_Wage', title='Min Wage in Actual Dollars')
).transform_filter(
    state_selection
))

fed_line = (alt.Chart(full_df).mark_line(color='grey', strokeDash=[4,1]).encode(
    x= 'Year:O',
    y = 'Federal_Minimum_Wage:Q'
))

texts = (alt.Chart(full_df).mark_text(dy=-170, size=16).encode(
    text= 'State:N',
).transform_filter(
    state_selection
))


line = fed_line + state_line + texts

state_min_wage = (choropleth & line)

state_min_wage

alt.VConcatChart(...)